<a href="https://colab.research.google.com/github/Hadjadj-Benakmoum/MobilityPredictionUsingML/blob/main/MarkovModel_test_Scenario_(1%2C2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Copy the dataset in the reuntime and unzip it

In [ ]:
#Copy/Unzip files from Google drive to accelrate the training 
#!cp -r '/content/drive/My Drive/movement.tar.gz' '/content/movement.tar.gz'
#!tar xvzf '/content/movement.tar.gz'

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
movement/2001-2003/00022d811ca3.log.mv
movement/2001-2003/00022d817c56.log.mv
movement/2001-2003/00022d819e70.log.mv
movement/2001-2003/00022d81b95c.log.mv
movement/2001-2003/00022d81c3cb.log.mv
movement/2001-2003/00022d81cc2f.log.mv
movement/2001-2003/00022d81d9ea.log.mv
movement/2001-2003/00022d81f225.log.mv
movement/2001-2003/00022d8205f0.log.mv
movement/2001-2003/00022d823726.log.mv
movement/2001-2003/00022d824820.log.mv
movement/2001-2003/00022d8272f1.log.mv
movement/2001-2003/00022d82bf5c.log.mv
movement/2001-2003/00022d82bf9b.log.mv
movement/2001-2003/00022d82d3e7.log.mv
movement/2001-2003/00022d82e062.log.mv
movement/2001-2003/00022d83b0a9.log.mv
movement/2001-2003/00022d83c57c.log.mv
movement/2001-2003/00022d83f767.log.mv
movement/2001-2003/00022d845798.log.mv
movement/2001-2003/00022d847258.log.mv
movement/2001-2003/00022d8482a0.log.mv
movement/2001-2003/00022d848912.log.mv
movement/2001-2003/00022d

# Load/split sequences functions

In [ ]:
def Load_Data(file):
  locations = []
  with open(file) as f:  
     for line in f:
         locations.append(line.split()[1]) 
  return locations

def Split_sequence(sequence,train_percentage):
  split_index = int(train_percentage * len(sequence))
  train_sequence = sequence[:split_index]
  test_sequence  = sequence[split_index:]
  return train_sequence,test_sequence

# Training function

In [ ]:
def train_markov_model(locations, train_percentage, w):
  #seq_size = len(locations)
  d = {}
  
  for i in range(len(locations)-w):
      context = ','.join(locations[i:i+w])
      next_location = locations[i+w]
      if(context in d.keys()):
          if(next_location in d[context].keys()):
              d[context][next_location]=d[context][next_location]+1
          else:
              d[context][next_location]=1
      else:
          d[context]={next_location:1}
  
  return d

# Predict function

## Breaking ties

In [ ]:
def most_frequent(List): 
    return max(set(List), key = List.count) 

Predict using breaking ties 

In [ ]:
def nxt_loc(model,test_loc,w,i,method):
  if (Built_context(i,w) in model.keys() ):
     next_locations_dic = model[test_loc]
     #print("next_loc_dict=",next_locations_dic)
     inverse = [(value, key) for value, key in next_locations_dic.items()]
     Max_val=(max(inverse)[1])
    # print ("conv list",inverse,"\n Max ",Max_val)
     list_of_max=[k for k,v in next_locations_dic.items() if v == Max_val]
    # print("list of max: ",list_of_max)
     if (len(list_of_max)>1):
      if (method == "first"):
        next_loc = locations[locations.index(list_of_max[0])]
      elif (method =="last"):
        next_loc = locations[locations.index(list_of_max[len(list_of_max)-1])]
      elif (method == "most"):
        next_loc=most_frequent(locations)
      return next_loc
     else:
      next_loc=max(next_locations_dic,key=next_locations_dic.get)
      return next_loc
  else:
    undefined_seq = Built_context(i,w)
    return undefined_seq

## build context

In [ ]:
def Built_context(index,w):
  test_loc=''
  for i in range(len(test_locations[index:(index+w-1)])):
    test_loc+=test_locations[index+i]+','
  test_loc += test_locations[index+w-1]
  #print(test_loc)
  return test_loc

# Test function


In [ ]:
def test (test,w, dictionary):
  correct=0
  incorrect=0
  undefined=0
  for i in range(len(test)-w):
    prediction = nxt_loc(dictionary,Built_context(i,w),w,i,method="first")
    real_location = test_locations[i+w]
   # print("predict= ", prediction, "|| Real Location = ",real_location, "||Context= ",Built_context(i,w)) 
    if (Built_context(i,w) in dictionary.keys() ):
      if (prediction == real_location ):
        correct +=1
      else:
        incorrect +=1
    else:
      undefined +=1
  accuracy = correct/(len(test))
  return correct,incorrect,accuracy,undefined

# Test Scenario 1 : Train/Test in each user

## Save result in CVS files

In [ ]:
#Test configuration
w=8
train_percentage=0.75
method = 'Markov'


#undefined_strategy = 'None'
Breaking_Ties = 'first'


path='/content/drive/My Drive/Mobilitytes/'


test_configuration ='Scenario:{0} W:{1} train_percentage:{2} Method:{3} Breaking_Ties:{4}.csv'.format(1,w,train_percentage,method,Breaking_Ties)
print('Test strategy : ', test_configuration)

Test strategy :  Scenario:1 W:8 train_percentage:0.75 Method:Markov Breaking_Ties:first.csv


In [ ]:
import csv
def save_results_csv(output_file,users_files,Corrects,Incorrects,Undefined,Accuracies):
  with open(output_file, 'w') as csvfile:
      header = ['users_files','Corrects','Incorrects','Undefined','Accuracies']
      writer = csv.writer(csvfile, delimiter=',')
      writer.writerow(header)
      outputs = zip(users_files,Corrects,Incorrects,Undefined,Accuracies)
      writer.writerows(outputs)

In [ ]:
accuracies=[]
Corrects=[]
Incorrects=[]
Undefined_Sequences=[]
train_percentage=0.75
w=8
path='/content/drive/My Drive/Mobilitytes/'
users_files =  os.listdir(path)
print('Number of files :',len(users_files))
for filename in users_files:
    if filename.endswith("mv"):
      #Read sequence
      locations = Load_Data(os.path.join(path,filename))

      #Split the sequence Train/Test
      train_locations,test_locations = Split_sequence(locations,train_percentage)
      d = train_markov_model(train_locations,train_percentage,w)
      c,i,a,undefined = test(test_locations,w,dictionary=d)
      accuracies.append(a)
      Corrects.append(c)
      Incorrects.append(i)
      Undefined_Sequences.append(undefined)
      print("Correct:",c)
      print("Incorrect",i)
      print("Accuracy",a)
      print("Undefined Sequence = ",undefined)
      print("======================================================================================================")
save_results_csv(test_configuration,users_files,Corrects,Incorrects,Undefined_Sequences,accuracies)
print(" Corrects= ",sum(Corrects))
print(" Incorrects= ",sum(Incorrects))
print("moyenne Acc= ",sum(accuracies)/len(accuracies))
print(" Undefined_Sequences= ",sum(Undefined_Sequences))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Correct: 21
Incorrect 11
Accuracy 0.14685314685314685
Undefined Sequence =  103
Correct: 5645
Incorrect 84
Accuracy 0.9717679462902393
Undefined Sequence =  72
Correct: 19
Incorrect 5
Accuracy 0.5
Undefined Sequence =  6
Correct: 0
Incorrect 0
Accuracy 0.0
Undefined Sequence =  116
Correct: 0
Incorrect 0
Accuracy 0.0
Undefined Sequence =  0
Correct: 273
Incorrect 10
Accuracy 0.8950819672131147
Undefined Sequence =  14
Correct: 0
Incorrect 0
Accuracy 0.0
Undefined Sequence =  19
Correct: 1136
Incorrect 17
Accuracy 0.9380677126341866
Undefined Sequence =  50
Correct: 566
Incorrect 113
Accuracy 0.6008492569002123
Undefined Sequence =  255
Correct: 11
Incorrect 10
Accuracy 0.3793103448275862
Undefined Sequence =  0
Correct: 54
Incorrect 9
Accuracy 0.22594142259414227
Undefined Sequence =  168
Correct: 13
Incorrect 3
Accuracy 0.02663934426229508
Undefined Sequence =  464
Correct: 20
Incorrect 7
Accuracy 0.02150537

# Test Scenario 2 : Train on users and Test in other users

In [ ]:
def train_on_many_users(train_files):

  global_models = None
  for filename in train_files:
    if filename.endswith("mv"):
      train_locations = Load_Data(os.path.join(path,filename))
      global_models = train_markov_model(train_locations,train_percentage=0.75,w=8) 
  return global_models 

In [ ]:
w=8
train_percentage=0.75
method = 'Markov'
Breaking_Ties = 'first'
path='/content/drive/My Drive/Mobilitytes/'
test_configuration ='Scenario:{0} W:{1} train_percentage:{2} Method:{3} Breaking_Ties:{4}.csv'.format(2,w,train_percentage,method,Breaking_Ties)
print('Test strategy : ', test_configuration)

Test strategy :  Scenario:2 W:8 train_percentage:0.75 Method:Markov Breaking_Ties:first.csv


In [ ]:
users_files =  os.listdir(path)
list_of_files=os.listdir(path)
train_files,test_files =  Split_sequence(list_of_files,0.75)


print('Start model training')
global_models = train_on_many_users(train_files)
print('End model training')

Accuracies=[]
Corrects=[]
Incorrects=[]
Undefined=[]

for filename in test_files :
    if filename.endswith("mv"):
      #Read sequence
      locations = Load_Data(os.path.join(path,filename))
      #Split the sequence Train/Test
      train_locations,test_locations = Split_sequence(locations,train_percentage)
     
      #Train the MARKOV model 
      models = train_markov_model(train_locations,train_percentage,w=4)

      #Test the Model 
      
      user_correct,user_incorrect,user_accuracy,user_undefined=test(test_locations,w=4,dictionary=models)                                                   
                                                                                              
                                                                                          
      
      #Collect results
      Corrects.append(user_correct)
      Incorrects.append(user_incorrect)
      Undefined.append(user_undefined)
      Accuracies.append(user_accuracy)
      
      #Display results
      print('User : ', filename)
      print('User lenght : ',len(locations))
      print('Correct : ',user_correct )
      print('Incorrect : ',user_incorrect)
      print('Undefined : ',user_undefined)
      print('Accuracy : ',user_accuracy)
      print("======================================================================================================")

#Save final results in csv file where the name contains information about the test (w,method ....). The name of file is : Configuration.csv

save_results_csv(test_configuration,users_files,Corrects,Incorrects,Undefined,Accuracies)

print('Global results : ')
print("======================================================================================================")
print('Corrects : ',sum(Corrects))
print('Incorrects : ',sum(Incorrects))
print('Undefined_Sequences : ',sum(Undefined))
print('Average Acc : ',sum(Accuracies)/len(Accuracies))



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Accuracy :  0.0
User :  260.mv
User lenght :  978
Correct :  211
Incorrect :  3
Undefined :  27
Accuracy :  0.8612244897959184
User :  157.mv
User lenght :  2166
Correct :  437
Incorrect :  73
Undefined :  28
Accuracy :  0.8062730627306273
User :  641.mv
User lenght :  10819
Correct :  1429
Incorrect :  1164
Undefined :  108
Accuracy :  0.5282809611829945
User :  768.mv
User lenght :  3904
Correct :  818
Incorrect :  104
Undefined :  50
Accuracy :  0.8381147540983607
User :  105.mv
User lenght :  36
Correct :  0
Incorrect :  0
Undefined :  5
Accuracy :  0.0
User :  276.mv
User lenght :  23764
Correct :  3789
Incorrect :  1446
Undefined :  702
Accuracy :  0.6377714189530382
User :  292.mv
User lenght :  15359
Correct :  3199
Incorrect :  613
Undefined :  24
Accuracy :  0.8330729166666667
User :  298.mv
User lenght :  1388
Correct :  164
Incorrect :  138
Undefined :  41
Accuracy :  0.47262247838616717
User :  5